In [17]:
from bs4 import BeautifulSoup
from bs4.builder import TreeBuilder
from bs4 import UnicodeDammit
import requests
import lxml
import time
import os
import io
import json
import core_constants as cc
import string
import csv
import cleantext as clean
import pandas as pd
import sqlite3 as sql
import traceback
import recordlinkage
from recordlinkage.base import BaseCompareFeature
import functions as fx

In [ ]:
years = cc.get_defYears()
headers= cc.get_header()
schoolsList = cc.get_schoolsList()
dataset = 'NFL'

In [ ]:
def queryBuilderFM(KeyDataSet, DataSet):
    fuzzyFields = cc.fuzzyFields[DataSet]
    query = '''SELECT '''
    count = len(fuzzyFields)
    i = 1
    for field in fuzzyFields:
        if i == count:
            query = query + field
        else:
           query = query + field + ', '
        i = i + 1
    if (KeyDataSet == 2):
        query = query + ''' FROM UnlinkedRivals '''
    elif (KeyDataSet == 3):
        query = query + ''' FROM UnlinkedNFL '''
    elif (KeyDataSet == 4):
        query = query + ''' FROM UnlinkedAllConference '''
    elif (KeyDataSet == 5):
        query = query + ''' FROM UnlinkedNCAA '''
    elif (KeyDataSet == 6):
        query = query + ''' FROM UnlinkedAllAmerican '''
    else:
        query = query + ''' FROM SourcedPlayers where KeyDataSet = ''' + str(KeyDataSet)
    return query

In [ ]:
def doFuzzyMatching (source, target):
    

    # Create Dataframes

    ###### Source Dataframe
    ######## I'm setting the key equal to the target dataset - that way we don't compare IDYR to ID
    ######## HEY when you changed target vs source being 247, I think you broke the linked players view.  those joins need to be revisited!
    key = fx.getKeyDataset(source)
    #tkey = getKeyDataset(target)
    SQL = fx.queryBuilderFM(key, source)
    if (key in (1,2)):
        df_source = (connDBAndReturnDF(SQL)).set_index('IDYR')
        df_source.index.name = source + '_IDYR'
        df_source['IDYR'] = df_source.index.get_level_values(0)
    else:
        df_source = (connDBAndReturnDF(SQL)).set_index('ID')
        df_source.index.name = source + '_ID'
        df_source['ID'] = df_source.index.get_level_values(0)
    
    ###### Target Dataframe
    key = fx.getKeyDataset(target)

    SQL = queryBuilderFM(key, target)
    if (key in (1,2)):
        df_target = (connDBAndReturnDF(SQL)).set_index('IDYR')
        df_target.index.name = target + '_IDYR'
        df_target['IDYR'] = df_target.index.get_level_values(0)
    else:
        df_target = (connDBAndReturnDF(SQL)).set_index('ID')
        df_target.index.name = target + '_ID'
        df_target['ID'] = df_target.index.get_level_values(0)
    
    #Create Blockers & Build Candidate Links - !!changing blockers from target to source
    indexer = recordlinkage.BlockIndex(on=cc.blockers[source])
    candidate_links = indexer.index(df_source, df_target)
    
    targetFuzzy = cc.fuzzyFields[source]
    sumFields = []
    c = recordlinkage.Compare()
    if 'IDYR' in targetFuzzy:
        c.exact('IDYR', 'IDYR', label='IDYR')
        sumFields.append('IDYR')
    if 'ID' in targetFuzzy:
        c.string('ID', 'ID', label='ID')
        sumFields.append('ID')
    if 'PlayerName' in targetFuzzy:
        c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
        sumFields.append('PlayerName')
    if 'City' in targetFuzzy:
        c.string('City', 'City', label='City')
        sumFields.append('City')
    if 'State' in targetFuzzy:
        c.exact('State', 'State', label='State')
        sumFields.append('State')
    if 'HighSchool' in targetFuzzy:
        c.string('HighSchool', 'HighSchool', label='HighSchool')
        sumFields.append('HighSchool')
    if 'Position' in targetFuzzy:
        c.exact('Position', 'Position', label='Position', agree_value=.25)
        sumFields.append('Position')
    if 'Year' in targetFuzzy:
        c.exact('Year', 'Year', label='Year', agree_value=.25)
        sumFields.append('Year')

    try:
        features = c.compute(candidate_links, df_source, df_target)
    except KeyError as e:
        print(e)
    
    sum = 0
    for field in sumFields:
        sum = sum + features[field]

    features['sum'] = sum / len(sumFields)

    filteredList = []
    noMatch = []

    features.insert(0, 'sourceID', features.index.get_level_values(0))
    features.insert(1, 'targetID', features.index.get_level_values(1))

    for idx, data in features.groupby(level=0):
        data = data.loc[data['sum'].idxmax()]
        if (data['ID'] == 1):
            filteredList.append(data)
        #NFL was set to .72 threshold
        #NCAA was set to .41864
        #AllConf was set to .8347 and .75 for annotations
        #AllAmerican was set to .831 and .72 for annotations
        elif (data['ID'] != 1 and data['sum'] > .65):
        #elif (data['ID'] != 1):
            filteredList.append(data)
        else:
            noMatch.append(data)

    dfFinal = pd.DataFrame()
    dfFinal = dfFinal.append(filteredList)
    dfFinal.to_csv("results.csv")

    return dfFinal

In [1]:
from jinjasql import JinjaSql
j = JinjaSql()

In [39]:
template = '''
SELECT
        {{ Select | sqlsafe}}
    FROM
        SourcedPlayers as a
        INNER JOIN Positions as b
            ON a.Position = b.Position
    WHERE
        a.KeyDataSet = 1
        {% if Ranked %}
            and a.Rating247 <> 0
        {% else %}
            and a.Rating247 is null
        {% endif %}
'''



In [40]:
data = {'Select':['a.ID', 'a.IDYR'],'Ranked':True}

In [41]:
query, bind_params = j.prepare_query(template, data)

In [46]:
def get_dynamic_select(dimensions):
    '''
    Returns the sql computing the number of transactions,
    as well as the total and the average transaction amounts
    for the provided list of column names as dimensions.
    '''
    params = {
       'Select': '\n    , '.join(dimensions)
    }
    return apply_sql_template(template, params)

In [47]:
print(get_dynamic_select(['ID', 'IDYR']))

NameError: name 'apply_sql_template' is not defined

In [22]:
df_247 = (fx.connDBAndReturnDF(query)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

,ID,PlayerName,Year,StandardizedPosition,KeyPositionGroup
247_IDYR,,,,,
paulthurston_nebraska_2012,paulthurston_nebraska,paulthurston,2012,OL,1
gregmcmullen_nebraska_2012,gregmcmullen_nebraska,gregmcmullen,2012,DE,2
jordanwesterkamp_nebraska_2012,jordanwesterkamp_nebraska,jordanwesterkamp,2012,WR,1
imanicross_nebraska_2012,imanicross_nebraska,imanicross,2012,RB,1
tommyarmstrong_nebraska_2012,tommyarmstrong_nebraska,tommyarmstrong,2012,QB,1
...,...,...,...,...,...
dondrellharris_troy_2013,dondrellharris_troy,dondrellharris,2013,QB,1
jordanchunn_troy_2013,jordanchunn_troy,jordanchunn,2013,FB,1
bryanslater_troy_2013,bryanslater_troy,bryanslater,2013,DE,2
